# Imports

In [1]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=710442b62e364c5442c9d69a68a2a912a62ec35aa201bf7c0825bdc54274f915
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.15.0
    Uninstalling emoji-2.15.0:
      Successfully uninstalled emoji-2.15.0


In [2]:
# Pytorch related
import torch
import torch.nn as nn

# Huggingface related
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from huggingface_hub import snapshot_download

# text preprocessing
from cleantext import clean
import re

# miscellanious
import math
import pandas as pd
import numpy as np
import json
import random

2025-11-04 06:40:17.251960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762238417.425046      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762238417.479042      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Reddit Domain Adaptation

## Model

In [3]:
model_checkpoint = "bert-base-uncased"
model_path = snapshot_download(repo_id=model_checkpoint, allow_patterns=["pytorch_model.bin", "config.json", "vocab.txt"])
model_path

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer

BertTokenizerFast(name_or_path='/root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
mask_token = tokenizer.mask_token
mask_token_id = tokenizer.mask_token_id
mask_token, mask_token_id

('[MASK]', 103)

In [7]:
inputs = tokenizer("I love [MASK] food", return_tensors="pt")
outputs = model(**inputs)
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
logits = outputs.logits[0, mask_token_index, :]
predicted_token_id = logits.argmax(dim=-1)
print(tokenizer.decode(predicted_token_id))

fast


In [8]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

## Dataset

In [9]:
dsr = load_dataset("Fredithefish/Reddit-TIFU")
dsr

tifu_collection.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/619 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'meta'],
        num_rows: 619
    })
})

In [10]:
dsr['train']['meta']

Column([{'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}])

In [11]:
dsr['train'][0]

{'title': 'TIFU by raising the flag upside down on a military base and causing local farmers to think the base was in distress.',
 'text': 'First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo 

In [12]:
dsr['train']['text']

Column(['First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo do any of these though, you had to go through Sea Cadet basic training camp, which is held at an Air National Guard base in Marseil

## Preprocessing

In [13]:
def preprocessing(ds):
    texts = ' '.join(ds['train']['text'])
    
    cleaned = clean(texts, lower=True, no_urls=True, no_punct=True, replace_with_url='')
    cleaned = re.sub(r'[\U0001f600-\U0001f64f\U0001f300-\U0001f5ff\U0001f680-\U0001f6ff\U0001f1e0-\U0001f1ff\U00002702-\U000027b0\U000024c2-\U0001f251]+', '', cleaned)
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(dsr)
len(corpus)

1988

In [14]:
print(type(corpus), len(corpus))
print(corpus[:3])

<class 'list'> 1988
['first things first i am not and never have been in the military this story concerns fifteen year old me when i was a sea cadet which is kind of like boy scouts but directly connected to the us navy wed wear the at the time blue camouflage us navy uniforms with a giant gold sign on our shirts and hats that made clear we werent actually in the navy like us being literal children wasnt a clue enough in the summer and school breaks we could go to camps you could go on an abbreviated naval diver course you could go to seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training you could go on a course to', 'learn to defensive drive humvees which i used as training hours for my learners permit i cant express in words how bizarre and amazing this group was when i was a kid to do any of these though you had to go through sea cadet basic training camp which is held at an air national guard base in marseilles pronounced marssalles i

In [15]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1590
    })
    test: Dataset({
        features: ['text'],
        num_rows: 398
    })
})

In [16]:
for i in range(3):
    print(ds['train'][i])

{'text': 'grits with a spicy pepper sauce so i go about preparing this while she is out i peel the ship and use the shells to make a stock for the grits i go and wash my hands not wanting my hands to smell like low tide i then take a few peppers and scrape the seeds out with my nails i then chop them add them to a skillet with melted butter and roast them and a few other more mild peppers for flavor toss in garlic corn and made this nice sauce next part i make her a nice plate roasted shrimp laid on the grits i begin pooring the sauce over her plate and she is coming in the door she sees the food and she'}
{'text': 'accidentally got pricked a little in my knuckle with a used needle through my latex glove it made a very very tiny dot that when i squeezed had blood come out a little the patients medical history is clear but who really knows right i poured alcohol on it and put on a bandaid my boss told me i was fine but after discussing with co workers i decided to go to the hospital the

## Guided Masking

In [17]:
!wget https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
!unzip NRC-Emotion-Lexicon.zip

--2025-11-04 06:40:41--  https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 25878449 (25M) [application/zip]
Saving to: ‘NRC-Emotion-Lexicon.zip’

NRC-Emotion-Lexicon 100%[===================>]  24.68M  27.5MB/s    in 0.9s    

2025-11-04 06:40:42 (27.5 MB/s) - ‘NRC-Emotion-Lexicon.zip’ saved [25878449/25878449]

Archive:  NRC-Emotion-Lexicon.zip
   creating: NRC-Emotion-Lexicon/
  inflating: NRC-Emotion-Lexicon/Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-ForVariousLanguages.txt  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: NRC-Emotion-Lexicon/Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: NRC-Emotion-Lexicon/README.txt  
  inflating: NRC-Emotion-Lexicon/Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
   creating: NRC-Emotion-Lexicon/OneFilePerLanguage/
  inflating: NRC-Emotion-Lexicon/ListOfLanguages-For-Which-Lexicon-Availabale.txt  
  inflating: NRC-Emotion-Lexicon/Paper2_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper2_NRC_Emotion_Lexicon.pdf  
   creating: NRC-Emotion-Lexicon/OneFil

In [18]:
filepath = '/kaggle/working/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
nrc = (
    pd.read_csv(filepath,
                names=["word", "emotion", "association"],
                sep='\t',
                skiprows=0,
                keep_default_na=False)
    .query("association == 1")
)
nrc

,word,emotion,association
19,abacus,trust,1
23,abandon,fear,1
25,abandon,negative,1
27,abandon,sadness,1
30,abandoned,anger,1
...,...,...,...
141461,zest,anticipation,1
141464,zest,joy,1
141466,zest,positive,1
141469,zest,trust,1


In [19]:
nrc['emotion'].unique()

array(['trust', 'fear', 'negative', 'sadness', 'anger', 'surprise',
       'positive', 'disgust', 'joy', 'anticipation'], dtype=object)

In [20]:
lexicon = nrc['word'].unique()
lexicon = set(map(lambda x : x.lower(), lexicon))
len(lexicon)

6453

In [21]:
def guided_masking(ds, mask_size=0.2, mask_token='[MASK]'):
    """
    Mask 20% of words in the text that appear in lexicon.
    If <20% masked, fill remainder with random masking.
    """
    text = ds["text"]
    words = re.findall(r"\w+|\S", text) 
    n_words = len([w for w in words if re.match(r"\w+", w)])
    
    if n_words == 0:
        ds["masked_text"] = text
        return ds
    
    # Mask positions where word is in lexicon
    maskable_indices = [i for i, w in enumerate(words) if w.lower() in lexicon]
    n_to_mask = max(1, int(mask_size * n_words))
    
    masked_indices = set(random.sample(maskable_indices, min(len(maskable_indices), n_to_mask)))
    
    # If not enough guided words, do random masking
    if len(masked_indices) < n_to_mask:
        remaining = n_to_mask - len(masked_indices)
        all_word_indices = [i for i, w in enumerate(words) if re.match(r"\w+", w)]
        available = list(set(all_word_indices) - masked_indices)
        random_indices = random.sample(available, min(remaining, len(available)))
        masked_indices.update(random_indices)

    masked_words = [mask_token if i in masked_indices else w for i, w in enumerate(words)]
    
    # Apply masking
    masked_text = ""
    for i, w in enumerate(masked_words):
        if i > 0 and re.match(r"\w+|\[MASK\]", w) and re.match(r"\w+|\[MASK\]", masked_words[i - 1]):
            masked_text += " "
        masked_text += w
    
    ds["masked_text"] = masked_text
    return ds

In [22]:
ds = ds.map(guided_masking)
ds

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 1590
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 398
    })
})

In [23]:
for i in range(3):
    print(ds['train'][i])

{'text': 'grits with a spicy pepper sauce so i go about preparing this while she is out i peel the ship and use the shells to make a stock for the grits i go and wash my hands not wanting my hands to smell like low tide i then take a few peppers and scrape the seeds out with my nails i then chop them add them to a skillet with melted butter and roast them and a few other more mild peppers for flavor toss in garlic corn and made this nice sauce next part i make her a nice plate roasted shrimp laid on the grits i begin pooring the sauce over her plate and she is coming in the door she sees the food and she', 'masked_text': 'grits [MASK] a spicy [MASK] sauce so i go about preparing this while she is [MASK] [MASK] peel [MASK] [MASK] and use the shells to make a stock [MASK] the grits i go and wash my hands not [MASK] my hands to [MASK] like [MASK] tide i then take a few peppers and scrape the [MASK] out [MASK] my nails i then [MASK] them add them to a skillet with melted butter and roast t

In [24]:
def tokenize_function(example):
    # Encode masked text as input, original text as label
    model_inputs = tokenizer(
        example["masked_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

In [25]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1590
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 398
    })
})

In [26]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/3033905897.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,3.480300,3.424353
2,3.029600,3.023394
3,2.744600,2.912018
4,2.555900,2.835855
5,2.446900,2.821465
6,2.319500,2.876262


TrainOutput(global_step=150, training_loss=2.8372973505655925, metrics={'train_runtime': 142.323, 'train_samples_per_second': 67.031, 'train_steps_per_second': 1.054, 'total_flos': 627743478528000.0, 'train_loss': 2.8372973505655925, 'epoch': 6.0})

In [27]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 17.75


## Testing

In [28]:
model_path = "/kaggle/working/bert-base-uncased-domain-adapted" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [29]:
def predict_masked_tokens(text, top_k=5):
    """
    Predict top-k tokens for each [MASK] in the text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    masked_indices = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    for idx in masked_indices:
        probs = predictions[0, idx].softmax(dim=0)
        topk = torch.topk(probs, k=top_k)
        tokens = tokenizer.convert_ids_to_tokens(topk.indices)
        scores = topk.values.tolist()

        print(f"\nMask position {idx.item()}:")
        for token, score in zip(tokens, scores):
            print(f"  {token:15s} | {score:.4f}")


In [30]:
text = "I have been feeling very [MASK] lately."
predict_masked_tokens(text)


Mask position 6:
  ill             | 0.2438
  bad             | 0.1420
  sick            | 0.1288
  uncomfortable   | 0.0577
  tired           | 0.0400


In [31]:
text = "I find it hard to [MASK] in activities I used to enjoy."
predict_masked_tokens(text)


Mask position 6:
  engage          | 0.4193
  participate     | 0.3680
  fit             | 0.0274
  relax           | 0.0149
  succeed         | 0.0140


In [32]:
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

def compare_models(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain_logits = model(**inputs).logits

    idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]
    base_probs = base_logits[0, idx].softmax(dim=0)
    domain_probs = domain_logits[0, idx].softmax(dim=0)

    print("\nBase BERT predictions:")
    for token, score in zip(*zip(*[(t, s) for t, s in zip(
        tokenizer.convert_ids_to_tokens(torch.topk(base_probs, 5).indices),
        torch.topk(base_probs, 5).values.tolist()
    )])):
        print(f"  {token:15s} | {score:.4f}")

    print("\nDomain-adapted predictions:")
    for token, score in zip(*zip(*[(t, s) for t, s in zip(
        tokenizer.convert_ids_to_tokens(torch.topk(domain_probs, 5).indices),
        torch.topk(domain_probs, 5).values.tolist()
    )])):
        print(f"  {token:15s} | {score:.4f}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
text = "I have been feeling very [MASK] lately."
compare_models(text)


Base BERT predictions:
  strange         | 0.0945
  restless        | 0.0817
  lonely          | 0.0795
  ill             | 0.0602
  sick            | 0.0602

Domain-adapted predictions:
  ill             | 0.2438
  bad             | 0.1420
  sick            | 0.1288
  uncomfortable   | 0.0577
  tired           | 0.0400


In [34]:
text = "Somedays I feel like [MASK]."
compare_models(text)


Base BERT predictions:
  it              | 0.0808
  this            | 0.0664
  crying          | 0.0657
  home            | 0.0614
  one             | 0.0578

Domain-adapted predictions:
  crying          | 0.3274
  shit            | 0.2632
  laughing        | 0.0371
  crazy           | 0.0233
  crap            | 0.0211


In [35]:
text = "I have been very [MASK] lately."
compare_models(text)


Base BERT predictions:
  busy            | 0.2860
  quiet           | 0.1167
  nervous         | 0.0385
  tired           | 0.0352
  worried         | 0.0319

Domain-adapted predictions:
  ill             | 0.1678
  busy            | 0.1171
  sick            | 0.0471
  nervous         | 0.0466
  quiet           | 0.0453


# Dataset for Mental Adaptation

In [36]:
dsm1 = load_dataset("ShreyaR/DepressionDetection")
dsm1

depression_dataset_reddit_twitter.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [37]:
dsm2 = load_dataset("hugginglearners/reddit-depression-cleaned")
dsm2

README.md: 0.00B [00:00, ?B/s]

depression_dataset_reddit_cleaned.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [38]:
dsm3 = load_dataset("jsfactory/mental_health_reddit_posts")
dsm3

posts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/24000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['body', 'subreddit'],
        num_rows: 24000
    })
})

In [39]:
dsm3['train'][0]

{'body': 'Thank you.\n\nMy son was recently diagnosed and I want to do whatever I can to support and help him. His current teacher also has ADHD and has been wonderful. I’m hopeful for my son.',
 'subreddit': 1}

In [40]:
merged_dsm = (
    list(dsm1["train"]["clean_text"]) +
    list(dsm2["train"]["clean_text"]) +
    list(dsm3["train"]["body"])
)

print(f"Total merged samples: {len(merged_dsm)}")

Total merged samples: 39462


In [41]:
def preprocessing(merged_dsm):
    texts = merged_dsm
    
    cleaned = clean(texts, lower=True, no_urls=True, no_punct=True, replace_with_url='')
    cleaned = re.sub(r'[\U0001f600-\U0001f64f\U0001f300-\U0001f5ff\U0001f680-\U0001f6ff\U0001f1e0-\U0001f1ff\U00002702-\U000027b0\U000024c2-\U0001f251]+', '', cleaned)
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(merged_dsm)
len(corpus)

17951

In [42]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 14360
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3591
    })
})

In [43]:
ds = ds.map(guided_masking)
ds

Map:   0%|          | 0/14360 [00:00<?, ? examples/s]

Map:   0%|          | 0/3591 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 14360
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 3591
    })
})

In [44]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/14360 [00:00<?, ? examples/s]

Map:   0%|          | 0/3591 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14360
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3591
    })
})

In [45]:
model_path = "/kaggle/working/bert-base-uncased-domain-adapted"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [46]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted-mental-dataset"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/4036430674.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.429100,2.252551
2,2.197100,2.166957
3,2.066600,2.195569
4,1.960500,2.194575
5,1.883200,2.249218
6,1.797200,2.310625


TrainOutput(global_step=1350, training_loss=2.0927287589179144, metrics={'train_runtime': 1268.8614, 'train_samples_per_second': 67.903, 'train_steps_per_second': 1.064, 'total_flos': 5669431667712000.0, 'train_loss': 2.0927287589179144, 'epoch': 6.0})

In [47]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 10.08


In [48]:
def compare_three_models(text, tokenizer, base_model, domain_model1, domain_model2):
    """
    Compare predictions of base BERT and two domain-adapted models on a masked text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain1_logits = domain_model1(**inputs).logits
        domain2_logits = domain_model2(**inputs).logits

    # Locate [MASK] token index
    idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]

    # Convert logits to probabilities
    base_probs = base_logits[0, idx].softmax(dim=0)
    domain1_probs = domain1_logits[0, idx].softmax(dim=0)
    domain2_probs = domain2_logits[0, idx].softmax(dim=0)

    def print_top(pred_probs, label):
        print(f"\n{label} predictions:")
        top = torch.topk(pred_probs, 5)
        for token_id, score in zip(top.indices, top.values):
            token = tokenizer.convert_ids_to_tokens(token_id.item())
            print(f"  {token:15s} | {score:.4f}")

    # Show results
    print("="*60)
    print(f"Input: {text}")
    print("="*60)

    print_top(base_probs, "Base BERT")
    print_top(domain1_probs, "Domain Adapted 1")
    print_top(domain2_probs, "Domain Adapted 2")

    print("="*60)

In [49]:
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
domain_adapted1 = AutoModelForMaskedLM.from_pretrained('/kaggle/working/bert-base-uncased-domain-adapted')
domain_adapted2 = AutoModelForMaskedLM.from_pretrained("/kaggle/working/bert-base-uncased-domain-adapted-mental-dataset")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/bert-base-uncased-domain-adapted")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
texts = [
    "I have been very [MASK] lately.",
    "Somedays I feel like [MASK].",
    "I have been feeling very [MASK] lately."
]

for text in texts:
    compare_three_models(text, tokenizer, base_model, domain_adapted1, domain_adapted2)

Input: I have been very [MASK] lately.

Base BERT predictions:
  busy            | 0.2860
  quiet           | 0.1167
  nervous         | 0.0385
  tired           | 0.0352
  worried         | 0.0319

Domain Adapted 1 predictions:
  ill             | 0.1678
  busy            | 0.1171
  sick            | 0.0471
  nervous         | 0.0466
  quiet           | 0.0453

Domain Adapted 2 predictions:
  depressed       | 0.3744
  anxious         | 0.1839
  quiet           | 0.0450
  lonely          | 0.0443
  nervous         | 0.0314
Input: Somedays I feel like [MASK].

Base BERT predictions:
  it              | 0.0808
  this            | 0.0664
  crying          | 0.0657
  home            | 0.0614
  one             | 0.0578

Domain Adapted 1 predictions:
  crying          | 0.3274
  shit            | 0.2632
  laughing        | 0.0371
  crazy           | 0.0233
  crap            | 0.0211

Domain Adapted 2 predictions:
  shit            | 0.6661
  crap            | 0.1014
  hell            | 0.02